
# Assignment 3 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 3, PROBLEM 1
Maximum Points = 8


Download the updated data folder from the course github website or just download directly the file [https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/smhi.csv](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/smhi.csv) from the github website and put it inside your data folder, i.e. you want the path `data/smhi.csv`. The data was aquired from SMHI (Swedish Meteorological and Hydrological Institute) and constitutes per hour measurements of wind in the Uppsala Aut station. The data consists of windspeed and direction. Your goal is to load the data and work with it a bit. The code you produce should load the file as it is, please do not alter the file as the autograder will only have access to the original file.

The file information is in Swedish so you need to use some translation service, for instance `Google translate` or ChatGPT.

1. [2p] Load the file, for instance using the `csv` package. Put the wind-direction as a numpy array and the wind-speed as another numpy array.
2. [2p] Use the wind-direction which is an angle in degrees and convert it into a point on the unit circle. Store the `x_coordinate` as one array and the `y_coordinate` as another. From these coordinates, construct the wind-velocity vector.
3. [2p] Calculate the average wind velocity and convert it back to direction and compare it to just taking average of the wind direction as given in the data-file.
4. [2p] The wind velocity is a $2$-dimensional random variable, calculate the empirical covariance matrix which should be a numpy array of shape (2,2).

For you to wonder about, is it more likely for you to have headwind or not when going to the university in the morning.

In [ ]:
import pandas as pd
import numpy as np

# Adjust this if you know exactly how many rows to skip
data = pd.read_csv('data/smhi.csv', delimiter=';', skiprows=11, on_bad_lines='skip', encoding='utf-8')

# Now extract wind direction and wind speed as numpy arrays
problem1_wind_direction = data['Vindriktning'].to_numpy()
problem1_wind_speed = data['Vindhastighet'].to_numpy()



In [3]:

# Convert degrees to radians
angles_rad = np.deg2rad(problem1_wind_direction)

# Calculate x and y components for wind direction
problem1_wind_direction_x_coordinate = np.cos(angles_rad)
problem1_wind_direction_y_coordinate = np.sin(angles_rad)

# Calculate x and y components of wind velocity vector
problem1_wind_velocity_x_coordinate = problem1_wind_speed * problem1_wind_direction_x_coordinate
problem1_wind_velocity_y_coordinate = problem1_wind_speed * problem1_wind_direction_y_coordinate


In [4]:

# Average x and y components of wind velocity
problem1_average_wind_velocity_x_coordinate = np.mean(problem1_wind_velocity_x_coordinate)
problem1_average_wind_velocity_y_coordinate = np.mean(problem1_wind_velocity_y_coordinate)

# Calculate the angle of the average wind velocity vector in degrees
problem1_average_wind_velocity_angle_degrees = np.rad2deg(
    np.arctan2(problem1_average_wind_velocity_y_coordinate, problem1_average_wind_velocity_x_coordinate)
)

# Calculate the average of the wind direction in degrees
problem1_average_wind_direction_angle_degrees = np.mean(problem1_wind_direction)


# Boolean check if the angles are approximately the same
problem1_same_angle = np.isclose(problem1_average_wind_velocity_angle_degrees, problem1_average_wind_direction_angle_degrees)


In [5]:

# Stack x and y components to form a 2D array for covariance calculation
wind_velocity_matrix = np.vstack((problem1_wind_velocity_x_coordinate, problem1_wind_velocity_y_coordinate))

# Calculate the covariance matrix
problem1_wind_velocity_covariance_matrix = np.cov(wind_velocity_matrix)



---
## Assignment 3, PROBLEM 2
Maximum Points = 8


For this problem you will need the [pandas](https://pandas.pydata.org/) package and the [sklearn](https://scikit-learn.org/stable/) package. Inside the `data` folder from the course website you will find a file called `indoor_train.csv`, this file includes a bunch of positions in (X,Y,Z) and also a location number. The idea is to assign a room number (Location) to the coordinates (X,Y,Z).

1. [2p] Take the data in the file `indoor_train.csv` and load it using pandas into a dataframe `df_train`
2. [3p] From this dataframe `df_train`, create two numpy arrays, one `Xtrain` and `Ytrain`, they should have sizes `(1154,3)` and `(1154,)` respectively. Their `dtype` should be `float64` and `int64` respectively.
3. [3p] Train a Support Vector Classifier, `sklearn.svc.SVC`, on `Xtrain, Ytrain` with `kernel='linear'` and name the trained model `svc_train`.

To mimic how [kaggle](https://www.kaggle.com/) works, the Autograder has access to a hidden test-set and will test your fitted model.

In [ ]:
import pandas as pd

file_path = 'data/indoor_train.csv'
df_train = pd.read_csv(file_path)
df_train.columns = df_train.columns.str.strip()


Position X  Position Y  Position Z  Location
0           32.0        15.0         4.4        18
1            8.0        17.0         1.5         9
2            4.0        13.0         4.4        13
3           39.0        16.0         4.4        18
4           34.0        12.0         7.6        15
...          ...         ...         ...       ...
1149        39.0         9.0         1.5        11
1150        38.0        26.0         4.4         6
1151         4.0         7.0         1.5         2
1152        21.0        19.0         1.5        17
1153        25.0        19.0         7.6        19

[1154 rows x 4 columns]

In [12]:
import numpy as np

Xtrain = df_train[['Position X', 'Position Y', 'Position Z']].to_numpy(dtype=np.float64)
Ytrain = df_train['Location'].to_numpy(dtype=np.int64)

Xtrain


array([[32. , 15. ,  4.4],
       [ 8. , 17. ,  1.5],
       [ 4. , 13. ,  4.4],
       ...,
       [ 4. ,  7. ,  1.5],
       [21. , 19. ,  1.5],
       [25. , 19. ,  7.6]])

In [13]:
from sklearn.svm import SVC

svc_train = SVC(kernel='linear')
svc_train.fit(Xtrain, Ytrain)


SVC(kernel='linear')

---
## Assignment 3, PROBLEM 3
Maximum Points = 8


Let us build a proportional model ($\mathbb{P}(Y=1 \mid X) = G(\beta_0+\beta \cdot X)$ where $G$ is the logistic function) for the spam vs not spam data. Here we assume that the features are presence vs not presence of a word, let $X_1,X_2,X_3$ denote the presence (1) or absence (0) of the words $("free", "prize", "win")$.

1. [2p] Load the file `data/spam.csv` and create two numpy arrays, `problem3_X` which has shape **(n_texts,3)** where each feature in `problem3_X` corresponds to $X_1,X_2,X_3$ from above, `problem3_Y` which has shape **(n_texts,)** and consists of a $1$ if the email is spam and $0$ if it is not. Split this data into a train-calibration-test sets where we have the split $40\%$, $20\%$, $40\%$, put this data in the designated variables in the code cell.

2. [2p] Follow the calculation from the lecture notes where we derive the logistic regression and implement the final loss function inside the class `ProportionalSpam`. You can use the `Test` cell to check that it gives the correct value for a test-point.

3. [2p] Train the model `problem3_ps` on the training data. The goal is to calibrate the probabilities output from the model. Start by creating a new variable `problem3_X_pred` (shape `(n_samples,1)`) which consists of the predictions of `problem3_ps` on the calibration dataset. Then train a calibration model using `sklearn.tree.DecisionTreeRegressor`, store this trained model in `problem3_calibrator`. Recall that calibration error is the following for a fixed function $f$
$$
    \sqrt{\mathbb{E}[|\mathbb{E}[Y \mid f(X)] - f(X)|^2]}.
$$

4. [2p] Use the trained model `problem3_ps` and the calibrator `problem3_calibrator` to make final predictions on the testing data, store the prediction in `problem3_final_predictions`. 

In [ ]:
# Load and preprocess the data
spam_data = pd.read_csv('data/spam.csv', encoding='latin1')
spam_data = spam_data.rename(columns={'v1': 'label', 'v2': 'message'})

# Encode the labels: ham = 0, spam = 1
spam_data['label'] = spam_data['label'].map({'ham': 0, 'spam': 1})

# Extract features: presence of specific words ('free', 'prize', 'win')
spam_data['free'] = spam_data['message'].str.contains('free', case=False, na=False).astype(int)
spam_data['prize'] = spam_data['message'].str.contains('prize', case=False, na=False).astype(int)
spam_data['win'] = spam_data['message'].str.contains('win', case=False, na=False).astype(int)

# Define the feature matrix and target array
problem3_X = spam_data[['free', 'prize', 'win']].to_numpy()
problem3_Y = spam_data['label'].to_numpy()



from sklearn.model_selection import train_test_split

# Split data into train (40%), calibration (20%), and test (40%) sets
X_temp, problem3_X_test, Y_temp, problem3_Y_test = train_test_split(
    problem3_X, problem3_Y, test_size=0.4, random_state=42)
problem3_X_train, problem3_X_calib, problem3_Y_train, problem3_Y_calib = train_test_split(
    X_temp, Y_temp, test_size=0.33, random_state=42)  # 33% of 60% = 20%


print(problem3_X_train.shape,problem3_X_calib.shape,problem3_X_test.shape,problem3_Y_train.shape,problem3_Y_calib.shape,problem3_Y_test.shape)

(2239, 3) (1104, 3) (2229, 3) (2239,) (1104,) (2229,)


In [58]:

from scipy import optimize

# Define the ProportionalSpam logistic regression class
class ProportionalSpam:
    def __init__(self):
        self.coeffs = None
        self.result = None

    def loss(self, X, Y, coeffs):
        preds = 1 / (1 + np.exp(-(np.dot(X, coeffs[1:]) + coeffs[0])))
        return -np.mean(Y * np.log(preds + 1e-9) + (1 - Y) * np.log(1 - preds + 1e-9))

    def fit(self, X, Y):
        initial_coeffs = np.zeros(X.shape[1] + 1)  # Initialize coefficients
        self.result = optimize.minimize(
            lambda coeffs: self.loss(X, Y, coeffs), initial_coeffs, method='cg')
        self.coeffs = self.result.x

    def predict(self, X):
        if self.coeffs is not None:
            preds = 1 / (1 + np.exp(-(np.dot(X, self.coeffs[1:]) + self.coeffs[0])))
            return preds
        else:
            raise ValueError("Model is not trained yet!")

In [59]:
# Train the logistic regression model
problem3_ps = ProportionalSpam()
problem3_ps.fit(problem3_X_train, problem3_Y_train)

# Predict probabilities for the calibration set
problem3_X_pred = problem3_ps.predict(problem3_X_calib)

from sklearn.tree import DecisionTreeRegressor

# Train the calibration model
problem3_calibrator = DecisionTreeRegressor()
problem3_calibrator.fit(problem3_X_pred.reshape(-1, 1), problem3_Y_calib)


DecisionTreeRegressor()

In [60]:
# Predict probabilities for the test set
test_probs = problem3_ps.predict(problem3_X_test)

# Use the calibration model to make final predictions
problem3_final_predictions = problem3_calibrator.predict(test_probs.reshape(-1, 1))

# Display the first few final predictions for verification
problem3_final_predictions[:10]


array([0.07917889, 0.07917889, 0.07917889, 0.07917889, 1.        ,
       0.07917889, 0.07917889, 0.07917889, 0.07917889, 0.07917889])

---
#### Local Test for Assignment 3, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [61]:
try:
    import numpy as np
    test_instance = ProportionalSpam()
    test_loss = test_instance.loss(np.array([[1,0,1],[0,1,1]]),np.array([1,0]),np.array([1.2,0.4,0.3,0.9]))
    assert (np.abs(test_loss-1.2828629432232497) < 1e-6)
    print("Your loss was correct for a test point")
except:
    print("Your loss was not correct on a test point")

Your loss was correct for a test point
